In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('gfl20-21data.txt')
winter2 = pd.read_csv ('gfl21-22data.txt')
winter3 = pd.read_csv ('gfl22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
precip_days2

,station,datetime,precip(mm),wxcodes
0,GFL,2021-12-01 00:16:00,T,-SN BR
1,GFL,2021-12-01 00:29:00,T,-SN BR
2,GFL,2021-12-01 00:35:00,T,-SN BR
3,GFL,2021-12-01 00:53:00,T,-SN BR
7,GFL,2021-12-01 03:34:00,T,-SN BR
...,...,...,...,...
3018,GFL,2022-02-27 22:14:00,T,-SN
3020,GFL,2022-02-27 23:07:00,T,-SN
3021,GFL,2022-02-27 23:29:00,T,-SN
3022,GFL,2022-02-27 23:32:00,T,-SN


In [7]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [8]:
precip_days3[68:70]

,station,datetime,precip(mm),wxcodes
219,GFL,2022-12-07 20:11:00,0.0001,-RA
220,GFL,2022-12-07 20:50:00,1.02,-RA BR


In [9]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [10]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [11]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [12]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [13]:
gfl = LocationInfo(
    'Osewego', 'New York', 
    latitude= 43.3095,
    longitude= -73.6440, 
    timezone='America/New_York'
)

gfl

LocationInfo(name='Osewego', region='New York', timezone='America/New_York', latitude=43.3095, longitude=-73.644)

In [14]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(gfl.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [15]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(gfl.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [16]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(gfl.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [17]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:07:53  2020-12-01 16:19:34  2020-12-01
1   2020-12-02 07:08:57  2020-12-02 16:19:17  2020-12-02
2   2020-12-03 07:10:00  2020-12-03 16:19:02  2020-12-03
3   2020-12-04 07:11:02  2020-12-04 16:18:50  2020-12-04
4   2020-12-05 07:12:02  2020-12-05 16:18:41  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:38:44  2021-02-24 17:37:19  2021-02-24
86  2021-02-25 06:37:08  2021-02-25 17:38:36  2021-02-25
87  2021-02-26 06:35:31  2021-02-26 17:39:53  2021-02-26
88  2021-02-27 06:33:53  2021-02-27 17:41:10  2021-02-27
89  2021-02-28 06:32:14  2021-02-28 17:42:26  2021-02-28

[90 rows x 3 columns]


In [18]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:07:37  2021-12-01 16:19:38  2021-12-01
1   2021-12-02 07:08:42  2021-12-02 16:19:20  2021-12-02
2   2021-12-03 07:09:45  2021-12-03 16:19:05  2021-12-03
3   2021-12-04 07:10:47  2021-12-04 16:18:53  2021-12-04
4   2021-12-05 07:11:48  2021-12-05 16:18:42  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:39:07  2022-02-24 17:37:00  2022-02-24
86  2022-02-25 06:37:31  2022-02-25 17:38:17  2022-02-25
87  2022-02-26 06:35:54  2022-02-26 17:39:34  2022-02-26
88  2022-02-27 06:34:17  2022-02-27 17:40:51  2022-02-27
89  2022-02-28 06:32:38  2022-02-28 17:42:07  2022-02-28

[90 rows x 3 columns]


In [19]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:07:21  2022-12-01 16:19:42  2022-12-01
1   2022-12-02 07:08:26  2022-12-02 16:19:24  2022-12-02
2   2022-12-03 07:09:30  2022-12-03 16:19:08  2022-12-03
3   2022-12-04 07:10:32  2022-12-04 16:18:55  2022-12-04
4   2022-12-05 07:11:33  2022-12-05 16:18:44  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:39:30  2023-02-24 17:36:41  2023-02-24
86  2023-02-25 06:37:54  2023-02-25 17:37:59  2023-02-25
87  2023-02-26 06:36:18  2023-02-26 17:39:16  2023-02-26
88  2023-02-27 06:34:40  2023-02-27 17:40:33  2023-02-27
89  2023-02-28 06:33:02  2023-02-28 17:41:49  2023-02-28

[90 rows x 3 columns]


In [20]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [21]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GFL,2020-12-01 00:53:00,2.0300,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
1,GFL,2020-12-01 01:53:00,3.3000,+RA BR,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
2,GFL,2020-12-01 02:53:00,3.0500,-RA BR,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
3,GFL,2020-12-01 03:53:00,0.2500,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
4,GFL,2020-12-01 04:53:00,2.7900,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
...,...,...,...,...,...,...,...
212,GFL,2021-02-27 11:53:00,2.0300,-SN FZFG,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10
213,GFL,2021-02-27 12:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10
214,GFL,2021-02-27 13:53:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10
215,GFL,2021-02-27 14:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10


In [22]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GFL,2021-12-01 00:53:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:07:37,2021-12-01 16:19:38
1,GFL,2021-12-01 03:53:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:07:37,2021-12-01 16:19:38
2,GFL,2021-12-01 04:53:00,0.0001,-SN BR,2021-12-01,2021-12-01 07:07:37,2021-12-01 16:19:38
3,GFL,2021-12-02 07:53:00,0.5100,-SN BR,2021-12-02,2021-12-02 07:08:42,2021-12-02 16:19:20
4,GFL,2021-12-02 08:53:00,1.2700,-SN BR,2021-12-02,2021-12-02 07:08:42,2021-12-02 16:19:20
...,...,...,...,...,...,...,...
234,GFL,2022-02-25 18:53:00,0.7600,-SN BR,2022-02-25,2022-02-25 06:37:31,2022-02-25 17:38:17
235,GFL,2022-02-25 19:53:00,0.2500,-SN BR,2022-02-25,2022-02-25 06:37:31,2022-02-25 17:38:17
236,GFL,2022-02-25 20:53:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:37:31,2022-02-25 17:38:17
237,GFL,2022-02-25 21:53:00,0.0001,-SN BR,2022-02-25,2022-02-25 06:37:31,2022-02-25 17:38:17


In [23]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GFL,2022-12-01 11:53:00,0.0001,-SN,2022-12-01,2022-12-01 07:07:21,2022-12-01 16:19:42
1,GFL,2022-12-03 04:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:09:30,2022-12-03 16:19:08
2,GFL,2022-12-03 07:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:09:30,2022-12-03 16:19:08
3,GFL,2022-12-03 10:53:00,0.2500,-RA,2022-12-03,2022-12-03 07:09:30,2022-12-03 16:19:08
4,GFL,2022-12-03 12:53:00,0.2500,-RA BR,2022-12-03,2022-12-03 07:09:30,2022-12-03 16:19:08
...,...,...,...,...,...,...,...
264,GFL,2023-02-25 20:53:00,0.0001,-SN BR,2023-02-25,2023-02-25 06:37:54,2023-02-25 17:37:59
265,GFL,2023-02-25 21:53:00,0.2500,-SN BR,2023-02-25,2023-02-25 06:37:54,2023-02-25 17:37:59
266,GFL,2023-02-25 22:53:00,0.0001,-SN BR,2023-02-25,2023-02-25 06:37:54,2023-02-25 17:37:59
267,GFL,2023-02-25 23:53:00,0.0001,-SN BR,2023-02-25,2023-02-25 06:37:54,2023-02-25 17:37:59


In [24]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
5,GFL,2020-12-01 15:53:00,2.2900,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
10,GFL,2020-12-02 09:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:57,2020-12-02 16:19:17
11,GFL,2020-12-02 10:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:57,2020-12-02 16:19:17
16,GFL,2020-12-05 13:53:00,0.2500,-RA,2020-12-05,2020-12-05 07:12:02,2020-12-05 16:18:41
17,GFL,2020-12-05 14:53:00,0.0001,-RA,2020-12-05,2020-12-05 07:12:02,2020-12-05 16:18:41
...,...,...,...,...,...,...,...
211,GFL,2021-02-27 10:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10
212,GFL,2021-02-27 11:53:00,2.0300,-SN FZFG,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10
213,GFL,2021-02-27 12:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10
214,GFL,2021-02-27 13:53:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10


In [25]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [26]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
10,GFL,2020-12-02 09:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:57,2020-12-02 16:19:17,2020-12-02 08:08:57,2020-12-02 15:19:17
11,GFL,2020-12-02 10:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:57,2020-12-02 16:19:17,2020-12-02 08:08:57,2020-12-02 15:19:17
16,GFL,2020-12-05 13:53:00,0.2500,-RA,2020-12-05,2020-12-05 07:12:02,2020-12-05 16:18:41,2020-12-05 08:12:02,2020-12-05 15:18:41
17,GFL,2020-12-05 14:53:00,0.0001,-RA,2020-12-05,2020-12-05 07:12:02,2020-12-05 16:18:41,2020-12-05 08:12:02,2020-12-05 15:18:41
19,GFL,2020-12-09 12:53:00,0.0001,-SN BR,2020-12-09,2020-12-09 07:15:49,2020-12-09 16:18:27,2020-12-09 08:15:49,2020-12-09 15:18:27
...,...,...,...,...,...,...,...,...,...
211,GFL,2021-02-27 10:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10,2021-02-27 07:33:53,2021-02-27 16:41:10
212,GFL,2021-02-27 11:53:00,2.0300,-SN FZFG,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10,2021-02-27 07:33:53,2021-02-27 16:41:10
213,GFL,2021-02-27 12:53:00,0.7600,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10,2021-02-27 07:33:53,2021-02-27 16:41:10
214,GFL,2021-02-27 13:53:00,0.0001,-SN BR,2021-02-27,2021-02-27 06:33:53,2021-02-27 17:41:10,2021-02-27 07:33:53,2021-02-27 16:41:10


In [27]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [28]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [29]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.7501814285714286
2021-2022: 0.5500448717948719
2022-2023: 0.9664926470588234


In [30]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,GFL,2020-12-01 00:53:00,2.0300,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
1,GFL,2020-12-01 01:53:00,3.3000,+RA BR,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
2,GFL,2020-12-01 02:53:00,3.0500,-RA BR,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
3,GFL,2020-12-01 03:53:00,0.2500,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
4,GFL,2020-12-01 04:53:00,2.7900,-RA,2020-12-01,2020-12-01 07:07:53,2020-12-01 16:19:34
...,...,...,...,...,...,...,...
207,GFL,2021-02-22 22:53:00,0.5100,-SN BR,2021-02-22,2021-02-22 06:41:54,2021-02-22 17:34:44
208,GFL,2021-02-22 23:53:00,0.0001,-SN BR,2021-02-22,2021-02-22 06:41:54,2021-02-22 17:34:44
209,GFL,2021-02-23 19:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 06:40:19,2021-02-23 17:36:02
210,GFL,2021-02-23 20:53:00,0.0001,-SN BR,2021-02-23,2021-02-23 06:40:19,2021-02-23 17:36:02


In [31]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_20684\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [32]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [33]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [34]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.35139000000000004
2021-2022: 0.5290563380281692
2022-2023: 0.48562795698924743


In [35]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('gfl.csv', index=False)